# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 27 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
!nvidia-smi

Tue Feb 14 15:57:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torchsummaryX wandb --quiet
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

NotImplementedError: ignored

In [ ]:
!mkdir /root/.kaggle
with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"","key":""}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle competitions download -c 11-785-s23-hw1p2 --force

100% 16.0G/16.0G [14:17<00:00, 18.9MB/s]
100% 16.0G/16.0G [14:17<00:00, 20.0MB/s]


In [ ]:
!mkdir '/content/data'

!unzip '/content/11-785-s23-hw1p2.zip' -d '/content/data'

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/11-785-s23-hw1p2/train-clean-360/transcript/8643-298549-0050.npy  
  inflating: /content/data/11-785-s23-hw1p2/train-clean-360/transcript/8643-298549-0051.npy  
  inflating: /content/data/11-785-s23-hw1p2/train-clean-360/transcript/8643-298549-0052.npy  
  inflating: /content/data/11-785-s23-hw1p2/train-clean-360/transcript/8643-298549-0053.npy  
  inflating: /content/data/11-785-s23-hw1p2/train-clean-360/transcript/8643-298549-0054.npy  
  inflating: /content/data/11-785-s23-hw1p2/train-clean-360/transcript/8643-298549-0055.npy  
  inflating: /content/data/11-785-s23-hw1p2/train-clean-360/transcript/8643-298549-0056.npy  
  inflating: /content/data/11-785-s23-hw1p2/train-clean-360/transcript/8643-298549-0057.npy  
  inflating: /content/data/11-785-s23-hw1p2/train-clean-360/transcript/8643-298549-0058.npy  
  inflating: /content/data/11-785-s23-hw1p2/train-clean-360/transcript/8643-298549-0059.npy  
  inflati

In [ ]:
!mkdir 'pretrained-model'
!kaggle kernels output krishrana1/checkpoint-learning -p 'pretrained-model'

Output file downloaded to pretrained-model/final-epoch-save.pth
Output file downloaded to pretrained-model/hw1p2.log
Output file downloaded to pretrained-model/model_arch.txt
Output file downloaded to pretrained-model/network4-e5-b2048.pth
Output file downloaded to pretrained-model/network4-resume-after-ckpt-2048.pth
Output file downloaded to pretrained-model/wandb/run-20230205_082039-354nlv8a/files/conda-environment.yaml
Output file downloaded to pretrained-model/wandb/run-20230205_082039-354nlv8a/files/config.yaml
Output file downloaded to pretrained-model/wandb/run-20230205_082039-354nlv8a/files/output.log
Output file downloaded to pretrained-model/wandb/run-20230205_082039-354nlv8a/files/requirements.txt
Output file downloaded to pretrained-model/wandb/run-20230205_082039-354nlv8a/files/wandb-metadata.json
Output file downloaded to pretrained-model/wandb/run-20230205_082039-354nlv8a/files/wandb-summary.json
Output file downloaded to pretrained-model/wandb/run-20230205_082039-354nlv

In [ ]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

In [ ]:
# Dataset class to load train and validation data
# /kaggle/input/11-785-s23-hw1p2/11-785-s23-hw1p2/train-clean-100/mfcc
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-360"): # Feel free to add more arguments
        T = 0
        self.context    = context
        self.phonemes   = phonemes
        
        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = root + "/" + partition + "/mfcc" 
        print(self.mfcc_dir)
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = root + "/" + partition + "/transcript" 

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = np.sort([i for i in os.listdir(self.mfcc_dir)]) 
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    = np.sort([i for i in os.listdir(self.transcript_dir)]) 
        

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names) 

        self.mfccs = []
        self.transcripts = []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(self.mfcc_dir + "/" + mfcc_names[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)
        #   Load the corresponding transcript
            transcript  = np.load(self.transcript_dir + "/" + transcript_names[i])
            transcript = transcript[1:-1]# Remove [SOS] and [EOS] from the transcript 
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)  
            #if i == int(len(mfcc_names) * 0.1):
            #    print(i)
            #    break
        print(mfcc.shape)
        print(transcript.shape)
        #print(self.transcripts)

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2) x 27, (T2+2) x 27 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that 
        # the final shape is T x 27 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs, axis=0)
        print("Before Padding: ", self.mfccs.shape)

        # TODO: Concatenate all transcripts in self.transcripts such that 
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts    = np.concatenate(self.transcripts, axis=0)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done. 
        # self.mfcc is an array of the format (Frames x Features). 
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. 
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        #zero_pad = np.zeros(shape=(1, self.mfccs.shape[1]))
        self.mfccs = np.pad(self.mfccs, pad_width=((self.context, self.context), (0, 0)))
        print(self.mfccs.shape)# TODO

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such. 
        # Hence, we map these phonemes to integers

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        print("Before: ", self.transcripts.shape)
        print(self.transcripts[554:560])
        phoneme_map = {ph:i for i, ph in enumerate(self.phonemes)}
        self.transcripts = np.array([phoneme_map[j] for j in self.transcripts])
        print(self.transcripts.shape)
        #print(self.transcripts[554:560])
        
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        #print("Ind: " + str(ind) + "Slice: " +  str(2*self.context + 1))
        frames = self.mfccs[ind: ind + 2*self.context + 1]
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data
        #print(frames.shape)

        frames      = torch.FloatTensor(frames) # Convert to tensors
        phonemes    = torch.tensor(self.transcripts[ind])       

        return frames, phonemes

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self, root, phonemes=PHONEMES, partition='test-clean'):
        self.test_dir = root + '/' + partition + '/mfcc'
        test_mfcc_names = np.sort([i for i in os.listdir(self.test_dir)])
        print(test_mfcc_names[:5])
        self.test_mfccs = []
        for i in range(len(test_mfcc_names)):
            test_mfcc = np.load(self.test_dir + '/' + test_mfcc_names[i])
            self.test_mfccs.append(test_mfcc)
        self.length = len(self.test_mfccs)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        return self.test_mfccs[idx]

    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

In [ ]:
config = {
    'epochs'        : 80,
    'batch_size'    : 2048,
    'context'       : 20,
    'init_lr'       : 1e-3,
    'lr_decay'      : 0.1,
    'dropout_prob_base' : 0.15,
    'dropout_prob_top' : 0.1,
    'scaler'        : torch.cuda.amp.GradScaler(enabled=True),
    'architecture'  : 'Network360'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

In [ ]:
#root = '/content/data/11-785-s23-hw1p2'
root = '/content/data/11-785-s23-hw1p2'
#context = 5

In [ ]:
#TODO: Create a dataset object using the AudioDataset class for the training data 
import time
st = time.time()
train_data = AudioDataset(root=root, context=config['context'], partition='train-clean-360') 
print("Time to load train-360 data: ", time.time() - st)
# TODO: Create a dataset object using the AudioDataset class for the validation data 
val_data = AudioDataset(root=root, context=config['context'], partition='dev-clean') 

/content/data/11-785-s23-hw1p2/train-clean-360/mfcc
(1079, 27)
(1079,)
Before Padding:  (130453995, 27)
(130454035, 27)
Before:  (130453995,)
['D' 'D' 'D' 'D' 'D' 'AW']
(130453995,)
Time to load train-360 data:  247.1544704437256
/content/data/11-785-s23-hw1p2/dev-clean/mfcc
(883, 27)
(883,)
Before Padding:  (1928204, 27)
(1928244, 27)
Before:  (1928204,)
['[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]']
(1928204,)


In [ ]:
# TODO: Create a dataset object using the AudioTestDataset class for the test data 
#test_data = AudioTestDataset(root=root, partition='test-clean') 

In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 4,
    batch_size  = config['batch_size'], 
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data, 
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

'''test_loader = torch.utils.data.DataLoader(
    dataset     = test_data, 
    num_workers = 2, 
    batch_size  = config['batch_size'], 
    pin_memory  = True, 
    shuffle     = False
)'''


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*27)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
#print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  2048
Context        :  20
Input size     :  1107
Output symbols :  42
Train dataset samples = 130453995, batches = 63699
Validation dataset samples = 1928204, batches = 942


In [ ]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([2048, 1107]) torch.Size([2048])


# Define Model, Loss Function and Optimizer

In [ ]:
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(Network, self).__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 2048),
            torch.nn.Dropout(config['dropout_prob_base']),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(),

            torch.nn.Linear(2048, 2048),
            torch.nn.Dropout(config['dropout_prob_base']),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(),

            torch.nn.Linear(2048, 2048),
            torch.nn.Dropout(config['dropout_prob_base']),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(),

            torch.nn.Linear(2048, 2048),
            torch.nn.Dropout(config['dropout_prob_base']),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(),

            torch.nn.Linear(2048, 2048),
            torch.nn.Dropout(config['dropout_prob_base']),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(),
            torch.nn.Linear(2048, output_size)
        ) 
        #self.model.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')

    def forward(self, x):
        out = self.model(x)

        return out

In [ ]:
#frames = torch.FloatTensor(np.zeros(shape=(2048, 1107)))
INPUT_SIZE  = (2*config['context'] + 1) * 27 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
ckpt = torch.load("/content/network360-ckpt.pth")
model.load_state_dict(ckpt['model_state_dict'])
summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

                         Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                                    
0_model.Linear_0         [1107, 2048]  [2048, 2048]  2.269184M  2.267136M
1_model.Dropout_1                   -  [2048, 2048]          -          -
2_model.BatchNorm1d_2          [2048]  [2048, 2048]     4.096k     2.048k
3_model.GELU_3                      -  [2048, 2048]          -          -
4_model.Linear_4         [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
5_model.Dropout_5                   -  [2048, 2048]          -          -
6_model.BatchNorm1d_6          [2048]  [2048, 2048]     4.096k     2.048k
7_model.GELU_7                      -  [2048, 2048]          -          -
8_model.Linear_8         [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
9_model.Dropout_9                   -  [2048, 2048]          -          -
10_model.BatchNorm1d_10        [2048]  [2048, 2048]     4.096k     2.048k
11_model.GELU_11                    - 

/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[1107, 2048]","[2048, 2048]",2269184.0,2267136.0
1_model.Dropout_1,-,"[2048, 2048]",NaN,NaN
2_model.BatchNorm1d_2,[2048],"[2048, 2048]",4096.0,2048.0
3_model.GELU_3,-,"[2048, 2048]",NaN,NaN
4_model.Linear_4,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0
5_model.Dropout_5,-,"[2048, 2048]",NaN,NaN
6_model.BatchNorm1d_6,[2048],"[2048, 2048]",4096.0,2048.0
7_model.GELU_7,-,"[2048, 2048]",NaN,NaN
8_model.Linear_8,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0


In [ ]:
print(model.model[0].weight)

Parameter containing:
tensor([[-0.2342, -0.1471, -0.2259,  ...,  0.0541,  0.0146, -0.0041],
        [ 0.0003,  0.0705, -0.0072,  ..., -0.0266,  0.0457,  0.1250],
        [ 0.1051,  0.0193, -0.0441,  ..., -0.0608, -0.2338, -0.2491],
        ...,
        [ 0.0432, -0.0243,  0.0031,  ...,  0.0023, -0.0370, -0.0261],
        [ 0.0823,  0.0455,  0.0298,  ...,  0.0489, -0.1050, -0.0101],
        [ 0.0028, -0.0404, -0.0063,  ..., -0.0183,  0.0612,  0.0117]],
       device='cuda:0', requires_grad=True)


In [ ]:
#lmbda = lambda epoch: 0.5

criterion = torch.nn.CrossEntropyLoss() # Defining Loss function. 

optimizer = torch.optim.AdamW(model.parameters(), lr = 0.0000156) #Defining Optimizer
#optimizer.load_state_dict(ckpt['optimizer_state_dict'])
# Recommended : Define Scheduler for Learning Rate, 
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
#scheduler2 = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda = lmbda)

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

30

In [ ]:
def train(model, dataloader, optimizer, criterion, scaler=config['scaler']):
    
    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    
    for i, (frames, phonemes) in enumerate(dataloader):
        
        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        ### Forward Propagation
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            logits  = model(frames)

            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        ### Backward Propagation
        scaler.scale(loss).backward() 
        
        ### Gradient Descent
        scaler.step(optimizer)
        scaler.update()

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))), 
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()
  
    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode(): 
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]
        
        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.07f}".format(float(vloss / (i + 1))), 
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()
    
        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

In [ ]:
wandb.login(key="") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name    = "colab-Network360-ckpt1-stepdrop", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "qrbmviyv", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2-krana", ### Project should be created in your wandb account 
    config  = config ### Wandb Config for your run
)

lr,▁
train_acc,▁
train_loss,▁
val_acc,▁
valid_loss,▁
lr,2e-05
train_acc,89.73111
train_loss,0.27916
val_acc,88.17248
valid_loss,0.39727


In [ ]:
### Save your model architecture as a string with str(model) 
model_arch  = str(model)

### Save it in a txt file 
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20230218_185906-ttzeikxs/files/model_arch.txt']

In [ ]:
import copy
#np.random.seed(42)
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")
min_val_acc = -float('inf')
save_path = 'network360-ckpt.pth'

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)
    
    if val_acc > min_val_acc:
        best_model = copy.deepcopy(model)
        torch.save({
            'epoch': epoch,
            'model_state_dict': best_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
            }, save_path)
        min_val_acc = val_acc
        print("Saving model with validation accuracy: ", val_acc)
    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run 
    # Optionally, you can log at each batch inside train/eval functions 
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss, 
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})
    scheduler.step(val_acc)

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best

### Finish your wandb run
run.finish()


Epoch 1/80


Train:   0%|          | 0/63699 [00:00<?, ?it/s]

Val:   0%|          | 0/942 [00:00<?, ?it/s]

Saving model with validation accuracy:  0.8817419428890617
	Train Acc 89.7407%	Train Loss 0.2789	 Learning Rate 0.0000156
	Val Acc 88.1742%	Val Loss 0.3952

Epoch 2/80


Train:   0%|          | 0/63699 [00:00<?, ?it/s]

...

Train:   0%|          | 0/63699 [00:00<?, ?it/s]

Val:   0%|          | 0/942 [00:00<?, ?it/s]

	Train Acc 89.7651%	Train Loss 0.2781	 Learning Rate 0.0000156
	Val Acc 88.1791%	Val Loss 0.3943

Epoch 8/80


Train:   0%|          | 0/63699 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), 'network4-2048-nodrop-train360.pth')

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self, root, phonemes=PHONEMES, partition='test-clean', context=20):
        self.test_dir = root + '/' + partition + '/mfcc'
        self.context = context
        test_mfcc_names = np.sort([i for i in os.listdir(self.test_dir)])
        print(test_mfcc_names[:5])
        self.test_mfccs = []
        for i in range(len(test_mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(self.test_dir + "/" + test_mfcc_names[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)
        #   Load the corresponding transcript
            #transcript  = np.load(self.transcript_dir + "/" + transcript_names[i])
            #transcript = transcript[1:-1]# Remove [SOS] and [EOS] from the transcript 
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.test_mfccs.append(mfcc)
            #if i == int(len(mfcc_names) * 0.1):
            #    print(i)
            #    break
        print(mfcc.shape)
        
        self.test_mfccs = np.concatenate(self.test_mfccs, axis=0)
        self.length = len(self.test_mfccs)
        print("Before Padding: ", len(self.test_mfccs))
        self.test_mfccs = np.pad(self.test_mfccs, pad_width=((self.context, self.context), (0, 0)))
        print("After padding: ", len(self.test_mfccs))
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, ind):
        frames = self.test_mfccs[ind: ind + 2*self.context + 1]
        frames = frames.flatten()
        frames      = torch.FloatTensor(frames)
        return frames

    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

In [ ]:
test_data = AudioTestDataset(root=root, partition='test-clean', context=20) 

['1089-134686-0000.npy' '1089-134686-0001.npy' '1089-134686-0002.npy'
 '1089-134686-0003.npy' '1089-134686-0004.npy']
(2052, 27)
Before Padding:  1934138
After padding:  1934178


In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data, 
    num_workers = 2, 
    batch_size  = config['batch_size'], 
    pin_memory  = True, 
    shuffle     = False
)

print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Test dataset samples = 1934138, batches = 945


In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []
    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)             
            
            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits, dim= 1)
            for j in predicted_phonemes:
                test_predictions.append(PHONEMES[j.item()])
            # TODO

    return test_predictions

In [ ]:
torch.cuda.empty_cache()
gc.collect()

1823

In [ ]:
predictions = test(model, test_loader)

  0%|          | 0/945 [00:00<?, ?it/s]

In [ ]:
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11-785-s23-hw1p2 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle

100% 19.3M/19.3M [00:04<00:00, 4.45MB/s]
Successfully submitted to Frame-Level Speech Recognition